In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from torch_geometric.nn import GCNConv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2, size3):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(1969, size1)
        self.conv2 = GCNConv(size1, size2)
        self.fc1 = torch.nn.Linear(size2, size3)
        self.fc2 = torch.nn.Linear(size3, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    size3 = trial.suggest_int("size3", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2, size3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [ ]:
for rs in range(10):
    
    # read data
    X_train = pd.read_csv("../../result/input_perturb_phyloP/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_test = pd.read_csv("../../result/input_perturb_phyloP/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    Y_train = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test_gene = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_perturb_phyloP/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_perturb_phyloP/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_thresholdAdaptive", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_K562", "CoExpr_ENCODE_K562_0.75", "CoExpr_GTEx_WholeBlood_0.75", "CoExpr_perturb_0.5", "Random_ER", "Random_SF", "NOTEARS_thresholdAdaptive"]:

        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_perturb_phyloP/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_perturb_phyloP/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"], best_params["size3"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_perturb_phyloP/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-11-30 21:16:52,418] A new study created in memory with name: no-name-7046aad7-30d4-4598-b7a4-b02f8b7e37c9
[I 2023-11-30 21:16:59,711] Trial 0 finished with value: 0.14502638578414917 and parameters: {'size1': 845, 'size2': 258, 'size3': 301, 'num_epochs': 119, 'lr': 0.006304823480990427}. Best is trial 0 with value: 0.14502638578414917.
[I 2023-11-30 21:17:00,064] Trial 1 finished with value: 0.1501331329345703 and parameters: {'size1': 142, 'size2': 267, 'size3': 329, 'num_epochs': 48, 'lr': 0.008191382778165933}. Best is trial 0 with value: 0.14502638578414917.
[I 2023-11-30 21:17:01,272] Trial 2 finished with value: 0.1555783599615097 and parameters: {'size1': 104, 'size2': 381, 'size3': 464, 'num_epochs': 169, 'lr': 0.009469992333794457}. Best is trial 0 with value: 0.14502638578414917.
[I 2023-11-30 21:17:01,529] Trial 3 finished with value: 0.1514125019311905 and parameters: {'size1': 732, 'size2': 49, 'size3': 471, 'num_epochs': 26, 'lr': 0.004077732076630911}. Best is t

[I 2023-11-30 21:17:31,876] Trial 3 finished with value: 0.15259025990962982 and parameters: {'size1': 85, 'size2': 269, 'size3': 412, 'num_epochs': 95, 'lr': 0.004068776906008546}. Best is trial 0 with value: 0.14993888139724731.
[I 2023-11-30 21:17:33,087] Trial 4 finished with value: 0.15583139657974243 and parameters: {'size1': 984, 'size2': 223, 'size3': 77, 'num_epochs': 124, 'lr': 0.008454912607265075}. Best is trial 0 with value: 0.14993888139724731.
[I 2023-11-30 21:17:33,116] Trial 5 pruned. 
[I 2023-11-30 21:17:33,174] Trial 6 pruned. 
[I 2023-11-30 21:17:33,225] Trial 7 pruned. 
[I 2023-11-30 21:17:33,280] Trial 8 pruned. 
[I 2023-11-30 21:17:33,343] Trial 9 pruned. 
[I 2023-11-30 21:17:33,398] Trial 10 pruned. 
[I 2023-11-30 21:17:33,479] Trial 11 pruned. 
[I 2023-11-30 21:17:33,570] Trial 12 pruned. 
[I 2023-11-30 21:17:33,644] Trial 13 pruned. 
[I 2023-11-30 21:17:33,947] Trial 14 pruned. 
[I 2023-11-30 21:17:34,028] Trial 15 pruned. 
[I 2023-11-30 21:17:34,119] Trial 16

[I 2023-11-30 21:17:56,364] Trial 4 finished with value: 0.15104974806308746 and parameters: {'size1': 408, 'size2': 7, 'size3': 94, 'num_epochs': 69, 'lr': 0.006373618345094054}. Best is trial 2 with value: 0.1486445516347885.
[I 2023-11-30 21:17:56,403] Trial 5 pruned. 
[I 2023-11-30 21:17:56,437] Trial 6 pruned. 
[I 2023-11-30 21:17:56,494] Trial 7 pruned. 
[I 2023-11-30 21:17:56,507] Trial 8 pruned. 
[I 2023-11-30 21:17:57,521] Trial 9 finished with value: 0.15196052193641663 and parameters: {'size1': 274, 'size2': 404, 'size3': 83, 'num_epochs': 135, 'lr': 0.0014903332762111385}. Best is trial 2 with value: 0.1486445516347885.
[I 2023-11-30 21:17:57,625] Trial 10 pruned. 
[I 2023-11-30 21:17:57,708] Trial 11 pruned. 
[I 2023-11-30 21:17:57,809] Trial 12 pruned. 
[I 2023-11-30 21:17:59,697] Trial 13 finished with value: 0.15304452180862427 and parameters: {'size1': 808, 'size2': 261, 'size3': 399, 'num_epochs': 190, 'lr': 0.0003212732181971786}. Best is trial 2 with value: 0.148644

[I 2023-11-30 21:18:23,332] Trial 1 finished with value: 0.1762876957654953 and parameters: {'size1': 239, 'size2': 335, 'size3': 105, 'num_epochs': 113, 'lr': 0.0053955504712360925}. Best is trial 0 with value: 0.15880636870861053.
[I 2023-11-30 21:18:28,307] Trial 2 finished with value: 0.19813883304595947 and parameters: {'size1': 1000, 'size2': 78, 'size3': 69, 'num_epochs': 180, 'lr': 0.009316652747809823}. Best is trial 0 with value: 0.15880636870861053.
[I 2023-11-30 21:18:28,706] Trial 3 finished with value: 0.15555180609226227 and parameters: {'size1': 154, 'size2': 405, 'size3': 209, 'num_epochs': 24, 'lr': 0.009801779118800709}. Best is trial 3 with value: 0.15555180609226227.
[I 2023-11-30 21:18:29,932] Trial 4 finished with value: 0.15565958619117737 and parameters: {'size1': 193, 'size2': 474, 'size3': 282, 'num_epochs': 67, 'lr': 0.007602451444017229}. Best is trial 3 with value: 0.15555180609226227.
[I 2023-11-30 21:18:30,786] Trial 5 pruned. 
[I 2023-11-30 21:18:30,833

[I 2023-11-30 21:19:34,241] Trial 7 pruned. 
[I 2023-11-30 21:19:34,314] Trial 8 pruned. 
[I 2023-11-30 21:19:34,397] Trial 9 pruned. 
[I 2023-11-30 21:19:34,489] Trial 10 pruned. 
[I 2023-11-30 21:19:34,581] Trial 11 pruned. 
[I 2023-11-30 21:19:34,682] Trial 12 pruned. 
[I 2023-11-30 21:19:34,772] Trial 13 pruned. 
[I 2023-11-30 21:19:34,869] Trial 14 pruned. 
[I 2023-11-30 21:19:34,965] Trial 15 pruned. 
[I 2023-11-30 21:19:35,081] Trial 16 pruned. 
[I 2023-11-30 21:19:35,151] Trial 17 pruned. 
[I 2023-11-30 21:19:35,256] Trial 18 pruned. 
[I 2023-11-30 21:19:35,339] Trial 19 pruned. 
[I 2023-11-30 21:19:39,047] A new study created in memory with name: no-name-7e7c1972-23e6-45c1-bf9b-6d922c1bd558
[I 2023-11-30 21:19:40,126] Trial 0 finished with value: 0.1561615765094757 and parameters: {'size1': 135, 'size2': 368, 'size3': 467, 'num_epochs': 71, 'lr': 0.00740529678242557}. Best is trial 0 with value: 0.1561615765094757.
[I 2023-11-30 21:19:40,545] Trial 1 finished with value: 0.155

...

[I 2023-11-30 22:10:22,253] Trial 3 finished with value: 0.1732785552740097 and parameters: {'size1': 817, 'size2': 165, 'size3': 345, 'num_epochs': 132, 'lr': 0.005662953447834944}. Best is trial 0 with value: 0.17306867241859436.
[I 2023-11-30 22:10:22,490] Trial 4 finished with value: 0.18847425282001495 and parameters: {'size1': 1010, 'size2': 443, 'size3': 157, 'num_epochs': 7, 'lr': 0.0011906886371291895}. Best is trial 0 with value: 0.17306867241859436.
[I 2023-11-30 22:10:22,502] Trial 5 pruned. 
[I 2023-11-30 22:10:22,527] Trial 6 pruned. 
[I 2023-11-30 22:10:22,583] Trial 7 pruned. 
[I 2023-11-30 22:10:22,629] Trial 8 pruned. 
[I 2023-11-30 22:10:22,654] Trial 9 pruned. 
[I 2023-11-30 22:10:23,940] Trial 10 finished with value: 0.1711394488811493 and parameters: {'size1': 655, 'size2': 7, 'size3': 24, 'num_epochs': 81, 'lr': 0.007755416866962207}. Best is trial 10 with value: 0.1711394488811493.
[I 2023-11-30 22:10:24,024] Trial 11 pruned. 
[I 2023-11-30 22:10:24,138] Trial 1

[I 2023-11-30 22:10:57,036] Trial 14 pruned. 
[I 2023-11-30 22:10:57,105] Trial 15 pruned. 
[I 2023-11-30 22:10:58,029] Trial 16 pruned. 
[I 2023-11-30 22:10:58,102] Trial 17 pruned. 
[I 2023-11-30 22:10:58,182] Trial 18 pruned. 
[I 2023-11-30 22:10:58,273] Trial 19 pruned. 
[I 2023-11-30 22:10:59,026] A new study created in memory with name: no-name-678ae31c-bcb7-4ac2-bd09-032894d2fae9
[I 2023-11-30 22:10:59,099] Trial 0 finished with value: 3.5102126598358154 and parameters: {'size1': 1011, 'size2': 364, 'size3': 8, 'num_epochs': 1, 'lr': 0.005190202015597002}. Best is trial 0 with value: 3.5102126598358154.
[I 2023-11-30 22:11:01,110] Trial 1 finished with value: 0.17971430718898773 and parameters: {'size1': 73, 'size2': 458, 'size3': 363, 'num_epochs': 181, 'lr': 0.001117187845616763}. Best is trial 1 with value: 0.17971430718898773.
[I 2023-11-30 22:11:03,486] Trial 2 finished with value: 0.18255969882011414 and parameters: {'size1': 629, 'size2': 188, 'size3': 12, 'num_epochs': 1

[I 2023-11-30 22:11:36,835] Trial 11 pruned. 
[I 2023-11-30 22:11:36,926] Trial 12 pruned. 
[I 2023-11-30 22:11:37,012] Trial 13 pruned. 
[I 2023-11-30 22:11:37,095] Trial 14 pruned. 
[I 2023-11-30 22:11:37,190] Trial 15 pruned. 
[I 2023-11-30 22:11:37,263] Trial 16 pruned. 
[I 2023-11-30 22:11:37,323] Trial 17 pruned. 
[I 2023-11-30 22:11:37,409] Trial 18 pruned. 
[I 2023-11-30 22:11:37,491] Trial 19 pruned. 
[I 2023-11-30 22:11:38,912] A new study created in memory with name: no-name-e9ac1e96-1cde-4bca-adab-55ecbd5da555
[I 2023-11-30 22:11:39,970] Trial 0 finished with value: 0.17582501471042633 and parameters: {'size1': 120, 'size2': 325, 'size3': 200, 'num_epochs': 134, 'lr': 0.0035213924662072108}. Best is trial 0 with value: 0.17582501471042633.
[I 2023-11-30 22:11:41,339] Trial 1 finished with value: 0.17061886191368103 and parameters: {'size1': 354, 'size2': 460, 'size3': 167, 'num_epochs': 126, 'lr': 0.009710386313750348}. Best is trial 1 with value: 0.17061886191368103.
[I 20

[I 2023-11-30 22:12:10,763] Trial 4 finished with value: 0.18070174753665924 and parameters: {'size1': 197, 'size2': 296, 'size3': 257, 'num_epochs': 20, 'lr': 0.0006869687063875286}. Best is trial 3 with value: 0.17121265828609467.
[I 2023-11-30 22:12:10,823] Trial 5 pruned. 
[I 2023-11-30 22:12:10,883] Trial 6 pruned. 
[I 2023-11-30 22:12:10,937] Trial 7 pruned. 
[I 2023-11-30 22:12:11,283] Trial 8 pruned. 
[I 2023-11-30 22:12:11,340] Trial 9 pruned. 
[I 2023-11-30 22:12:11,712] Trial 10 pruned. 
[I 2023-11-30 22:12:11,771] Trial 11 pruned. 
[I 2023-11-30 22:12:11,846] Trial 12 pruned. 
[I 2023-11-30 22:12:12,468] Trial 13 finished with value: 0.17024721205234528 and parameters: {'size1': 47, 'size2': 27, 'size3': 353, 'num_epochs': 77, 'lr': 0.009833930880239771}. Best is trial 13 with value: 0.17024721205234528.
[I 2023-11-30 22:12:13,946] Trial 14 finished with value: 0.17034326493740082 and parameters: {'size1': 421, 'size2': 9, 'size3': 339, 'num_epochs': 147, 'lr': 0.0087941562

[I 2023-11-30 22:12:46,732] Trial 19 pruned. 
[I 2023-11-30 22:12:48,491] A new study created in memory with name: no-name-dfad4dda-2dde-4465-87fa-88cb9fb049aa
[I 2023-11-30 22:12:52,416] Trial 0 finished with value: 0.17222735285758972 and parameters: {'size1': 590, 'size2': 188, 'size3': 282, 'num_epochs': 52, 'lr': 0.007694754740775057}. Best is trial 0 with value: 0.17222735285758972.
[I 2023-11-30 22:13:05,629] Trial 1 finished with value: 0.16873233020305634 and parameters: {'size1': 554, 'size2': 277, 'size3': 86, 'num_epochs': 170, 'lr': 0.009325836229060474}. Best is trial 1 with value: 0.16873233020305634.
[I 2023-11-30 22:13:18,848] Trial 2 finished with value: 0.17179103195667267 and parameters: {'size1': 625, 'size2': 392, 'size3': 455, 'num_epochs': 162, 'lr': 0.004278850662557853}. Best is trial 1 with value: 0.16873233020305634.
[I 2023-11-30 22:13:28,082] Trial 3 finished with value: 0.16847601532936096 and parameters: {'size1': 968, 'size2': 113, 'size3': 391, 'num_ep